In [1]:
import numpy as np
import pandas as pd

from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NIMBUS_EA import NIMBUS_RVEA, NIMBUS_NSGAIII
import plotly.express as ex
from pygmo import fast_non_dominated_sorting as nds
import plotly.graph_objects as go

In [2]:
problem = test_problem_builder(name="DTLZ2", n_of_objectives=3, n_of_variables=5)
problem.ideal = np.asarray([0, 0, 0])
problem.nadir = np.asarray([1, 1, 1])

In [4]:
evo_NSGA = NIMBUS_NSGAIII(problem, n_gen_per_iter=200)

In [101]:
plot, pref = evo_NSGA.requests()
print(pref.content["message"])

Provide a reference point worse than to the ideal point and better than the nadir point.
Ideal point: 
f1    0
f2    0
f3    0
Name: ideal, dtype: object
Nadir point: 
f1    1
f2    1
f3    1
Name: nadir, dtype: object
The reference point will be used to create scalarization functions in the preferred region.



In [102]:
response = np.asarray([[0.2,0.4, 0.2]])
pref.response = pd.DataFrame(response, columns=pref.content['dimensions_data'].columns)

In [103]:
plot, pref = evo_NSGA.iterate(pref)

In [104]:
fig = plot_dtlz2(plot, response)
fig

In [109]:
phi = np.linspace(0, np.pi/2, 50)
theta = np.linspace(0, np.pi/2, 50)
phi, theta = np.meshgrid(phi, theta)
x = np.cos(theta)*np.sin(phi)
y = np.sin(theta)*np.sin(phi)
z = np.cos(phi)

In [110]:
data = np.vstack((x.ravel(),y.ravel(),z.ravel(),)).T

In [129]:
scalar_data = np.asarray([scalar(data, response) for scalar in evo_NSGA.scalarization_methods]).T

In [138]:
fig_scala = go.Figure(go.Scatter3d(
    x=scalar_data[:,0],
    y=scalar_data[:,1],
    z=scalar_data[:,2],
    mode='markers',
    marker=dict(size=4),
    opacity=0.3,
    name="Whole true pareto front"
))

In [139]:
non_dom = nds(scalar_data)[0][0]

In [140]:
fig_scala.add_trace(
    go.Scatter3d(
        x=scalar_data[non_dom,0],
        y=scalar_data[non_dom,1],
        z=scalar_data[non_dom,2],
        mode = "markers",
        opacity=0.8,
        marker=dict(size=4),
        name="Non-dom from true pareto front"))

temp = np.asarray([scalar(evo_NSGA.population.objectives, response) for scalar in evo_NSGA.scalarization_methods]).T

fig_scala.add_trace(
    go.Scatter3d(
        x=temp[:,0],
        y=temp[:,1],
        z=temp[:,2],
        opacity=0.8,
        mode = "markers",
        name="NSGA-III",
        marker=dict(size=4)))

temp = np.asarray([scalar(evo_RVEA.population.objectives, response) for scalar in evo_NSGA.scalarization_methods]).T

fig_scala.add_trace(
    go.Scatter3d(
        x=temp[:,0],
        y=temp[:,1],
        z=temp[:,2],
        opacity=0.8,
        mode = "markers",
        name="RVEA",
        marker=dict(size=4)))

In [141]:
from plotly.offline.offline import plot

In [142]:
plot(fig_scala, "Scalarized_space.html")

'temp-plot.html'

In [119]:
fig = go.Figure(go.Surface(x=x, y=y, z=z, colorscale=["red", "red"], opacity=0.5))

In [120]:
fig.add_trace(go.Scatter3d(x=data[non_dom,0],y=data[non_dom,1],z=data[non_dom,2], marker=dict(size=4)))

In [64]:
evo_RVEA = NIMBUS_RVEA(problem, n_gen_per_iter=200)

In [143]:
plot, pref = evo_RVEA.requests()
print(pref.content["message"])

Provide a reference point worse than to the ideal point and better than the nadir point.
Ideal point: 
f1    0
f2    0
f3    0
Name: ideal, dtype: object
Nadir point: 
f1    1
f2    1
f3    1
Name: nadir, dtype: object
The reference point will be used to create scalarization functions in the preferred region.



In [144]:
pref.response = pd.DataFrame(response, columns=pref.content['dimensions_data'].columns)

In [145]:
plot, pref = evo_RVEA.iterate(pref)

In [148]:
plot_dtlz2(plot, response)

In [147]:
def plot_dtlz2(plot, response):
    # NIMBUSEA
    obj = plot.content['data'].values
    non_dom_id = nds(obj)[0][0]
    fig = go.Figure(
        go.Scatter3d(
            x=obj[:, 0],
            y=obj[:, 1],
            z=obj[:, 2],
            name="NIMBUS-EA",
            mode='markers',
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    #True Front
    phi = np.linspace(0, np.pi/2, 10)
    theta = np.linspace(0, np.pi/2, 10)
    phi, theta = np.meshgrid(phi, theta)
    x = np.cos(theta)*np.sin(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(phi)
    fig = fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale=["red", "red"], opacity=0.5))

    #Nadir Point
    fig.add_trace(
        go.Scatter3d(
            x=[1],
            y=[1],
            z=[1],
            name="Nadir Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )



    # Reference Point

    fig.add_trace(
        go.Scatter3d(
            x=[response[0,0]],
            y=[response[0,1]],
            z=[response[0,2]],
            name="Reference Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    # STOM

    fig.add_trace(
        go.Scatter3d(
            x=[0, response[0,0],100*response[0,0]],
            y=[0, response[0,1],100*response[0,1]],
            z=[0, response[0,2],100*response[0,2]],
            name="STOM",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )

    #ASF

    fig.add_trace(
        go.Scatter3d(
            x=[response[0,0]-1, response[0,0]+1],
            y=[response[0,1]-1, response[0,1]+1],
            z=[response[0,2]-1, response[0,2]+1],
            name="ASF",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )
    #AUG

    fig.add_trace(
        go.Scatter3d(
            x=[10*response[0,0] - 9*1,response[0,0], 1],
            y=[10*response[0,1] - 9*1,response[0,1], 1],
            z=[10*response[0,2] - 9*1,response[0,2], 1],
            name="AUG",
            marker=dict(
            size=2,
            opacity=0.8
        ),
            line=dict(width=10)
        )
    )

    # Ideal Point
    fig.add_trace(
        go.Scatter3d(
            x=[0],
            y=[0],
            z=[0],
            name="Ideal Point",
            marker=dict(
            size=4,
            opacity=0.8
        )
        )
    )

    fig.update_layout(
        scene = dict(
            xaxis = dict(range=[0,1.1],),
            yaxis = dict(range=[0,1.1],),
            zaxis = dict(range=[0,1.1],),
            aspectratio=dict(x=1,y=1,z=1))
                     )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    return fig